In [1]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

2019-08-12 17:36:45,721 - core.helpers.session_helper.SessionHelper - INFO - Creating session for dev environment...
2019-08-12 17:36:45,748 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating administrator mocks.
2019-08-12 17:36:45,788 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating administrator mocks.
2019-08-12 17:36:45,789 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating pharmaceutical company mocks.
2019-08-12 17:36:45,793 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating pharmaceutical company mocks.
2019-08-12 17:36:45,794 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating brand mocks.
2019-08-12 17:36:45,802 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Done generating brand mocks.
2019-08-12 17:36:45,803 - core.helpers.configuration_mocker.ConfigurationMocker - DEBUG - Generating segment mocks.
2019-08-12 17:36:4

In [2]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "accredo_cancel_before_active" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
input_pharma = "sun"
input_brand = "ilumya"
input_state = "ingest"
input_name = "symphony_health_association_ingest_column_mapping"
input_branch = "sun-extract-validation" # if None, input_branch is automagically set to your working branch

In [3]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
transform_id = ids[0]
run_id = ids[1]

2019-08-12 17:36:46,018 - core.logging - DEBUG - Adding/getting mocks for specified configurations...
2019-08-12 17:36:46,058 - core.logging - DEBUG - Done. Creating mock run event and committing results to configuration mocker.


In [4]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::accredo_cancel_before_active
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [5]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''
    # Column aliases
    status_date: str # Status date column
    ref_date: str # Referral date column
    patient: str # Patient ID column
    pharm: str # Pharmacy Name column
    product: str # Medication Name column
    status_date: str # Status Date column
    substatus_date: str # Substatus Date column
    ic_status: str # IntegriChain Status column
    ic_substatus: str # IntegriChain Substatus column
    pjh: str # Patient Journey Hierarchy column
        
    # Possible status values
    pending: str # Pending status 'PENDING'
    active: str # Active status 'ACTIVE'
    cancelled: str # Cancelled status 'CANCELLED'
    discontinued: str # Discontinued status 'DISCONTINUED'
        
    # Possible substatus values
    pending_new_substatus: str # New substatus when status is 'PENDING'
    shipped_substatus: str # Shipment substatus when status is 'ACTIVE'
    
    # Possible PJH values
    bvpa: str # BV/PA pjh
    intake: str # Intake pjh
    fulfillment: str # Fulfillment pjh
    transfer: str # Transferred pjh
    payer: str # Payer pjh

transform = Transform()

In [6]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"

transform.patient = 'msa_patient_id'
transform.pharm = 'pharm_code'
transform.product = 'medication'
transform.status

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

Replaces 

### Transformation

In [7]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch, state=input_state, parent=input_pharma, child=input_brand, dataset=input_name)
run_filter = []
run_filter.append(dict(partition="__metadata_run_id", comparison="==", values=[3]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!
final_dataframe = input_contract.fetch(filters=run_filter)

2019-08-12 17:36:46,656 - core.dataset_contract.DatasetContract - INFO - Fetching dataframe from s3 location s3://ichain-dev/sun-extract-validation/sun/ilumya/ingest/symphony_health_association_ingest_column_mapping.


In [9]:
final_dataframe.head()

,rec_date,pharm_code,pharm_npi,transtype,pharm_transaction_id,trans_seq,ref_source,ref_date,program_id,pharmacy_id,...,copay_assist_amount,oth_payer_amt,xfer_pharmname,msa_patient_id,msa_patient_bmap,__metadata_run_timestamp,__metadata_app_version,__metadata_output_contract,__metadata_transform_timestamp,__metadata_run_id
0,20181024115959,ACCREDO,1346208949,COM,279133432018102401,0,DIRECT,20181019120000,None,27913343,...,None,None,None,None,NNNNV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
1,20181025115959,ACCREDO,1346208949,COM,278370982018102502,0,DIRECT,20181022120000,None,27837098,...,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
2,20181029115959,ACCREDO,1346208949,COM,279181482018102903,0,DIRECT,20181024120000,None,27918148,...,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
3,20181102115959,ACCREDO,1346208949,COM,267244982018110204,0,DIRECT,20181030120000,None,26724498,...,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3
4,20181106115959,ACCREDO,1346208949,COM,160618142018110605,0,DIRECT,20181102120000,None,16061814,...,None,None,None,None,NNNVV,2019-07-01 13:25:07,0.0.11,s3://ichain-dev/sun-extract-validation/sun/ilu...,2019-07-01 13:35:22,3


In [13]:
final_dataframe.columns

Index(['rec_date', 'pharm_code', 'pharm_npi', 'transtype',
       'pharm_transaction_id', 'trans_seq', 'ref_source', 'ref_date',
       'program_id', 'pharmacy_id', 'pat_last_name', 'pat_first_name',
       'pat_dob', 'pat_gender', 'pat_addr1', 'pat_addr2', 'pat_city',
       'pat_state', 'pat_zip', 'dx1_code', 'dx2_code', 'status_date',
       'status_code', 'sub_status', 'pres_last_name', 'pres_first_name',
       'pres_addr1', 'pres_addr2', 'pres_city', 'pres_state', 'pres_zip',
       'pres_phone', 'pres_npi', 'pres_dea', 'facility_name', 'rxdate',
       'rxnumber', 'rxrefills', 'rxfill', 'refill_remaining', 'prev_disp',
       'rx_ndc_number', 'medication', 'quantity', 'day_supply', 'ship_date',
       'ship_carrier', 'shiptracking_num', 'ship_location', 'ship_address',
       'ship_city', 'ship_state', 'ship_zip', 'has_medical',
       'primary_coverage_type', 'primary_payer_name', 'primary_payer_type',
       'secondary_coverage_type', 'secondary_payer_name',
       'secondary_

In [11]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe

### Publish

In [12]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()

2019-08-12 17:36:54,176 - core.dataset_contract.DatasetContract - INFO - Publishing dataframe to s3 location s3://ichain-dev/ds301_accredo_enrichment/sun/ilumya/enrich/accredo_cancel_before_active with run ID 940808.
2019-08-12 17:36:54,188 - core.dataset_contract.DatasetContract - DEBUG - Publishing dataframe to Redshift Spectrum database ichain_core to schema.table                 data_core.sun_ilumya_accredo_cancel_before_active...
2019-08-12 17:36:54,552 - s3parq.publish_redshift - INFO - Running query to create schema: CREATE EXTERNAL SCHEMA IF NOT EXISTS data_core                 FROM DATA CATALOG                 database 'ichain_core'                 iam_role 'arn:aws:iam::265991248033:role/mySpectrumRole';


/usr/local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


ProgrammingError: (psycopg2.ProgrammingError) permission denied for database ichain_core

[SQL: CREATE EXTERNAL SCHEMA IF NOT EXISTS data_core                 FROM DATA CATALOG                 database 'ichain_core'                 iam_role 'arn:aws:iam::265991248033:role/mySpectrumRole';]
(Background on this error at: http://sqlalche.me/e/f405)